<a href="https://colab.research.google.com/github/KirkDCO/HandsOnML_Exercises/blob/main/Ch17_Q11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!cp -r drive/MyDrive/Colab\ Notebooks/HandsOnML/Ch17_Q10/flowers sample_data/.

In [3]:
# imports and globals

import tensorflow as tf
from tensorflow import keras
K = keras.backend

import numpy as np
import matplotlib.pyplot as plt
import random

BATCH_SIZE = 32 

IMG_WIDTH  = 96 
IMG_HEIGHT = 96 

ENCODING_SIZE = 8 

In [ ]:
# create a flower with labels generator
flower_generator = tf.keras.preprocessing.image_dataset_from_directory(
  "sample_data/flowers",
  seed = 84,
  image_size = (IMG_HEIGHT, IMG_WIDTH),
  batch_size = BATCH_SIZE, 
  labels = 'inferred'
)

# keep the flower names for future reference
flower_names = flower_generator.class_names

# create a training data generator 
training_generator = tf.keras.preprocessing.image_dataset_from_directory(
  "sample_data/flowers",
  seed = 42,
  image_size = (IMG_HEIGHT, IMG_WIDTH),
  batch_size = BATCH_SIZE, 
  labels = None, 
  validation_split = 0.2,
  subset = 'training'
)

# and a validation set generator
validation_generator = tf.keras.preprocessing.image_dataset_from_directory(
  "sample_data/flowers",
  seed = 42,
  image_size = (IMG_HEIGHT, IMG_WIDTH),
  batch_size = BATCH_SIZE, 
  labels = None, 
  validation_split = 0.2,
  subset = 'validation'
)

# autoencoders require the input matrix as both the input
# and the output
# provide input -> input for the autoencoder
# and, do scaling
def replicate_inputs_to_outputs(images):
  return images/255, images/255

# create prefetch generators to speed things up a bit
training_generator = training_generator.prefetch(128)
validation_generator = validation_generator.prefetch(128)

# create the final generators to be used in training the autoencoders
X_train = training_generator.map(replicate_inputs_to_outputs)
X_valid = validation_generator.map(replicate_inputs_to_outputs

In [ ]:
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))              # not shown in the book
        for X_batch in dataset:
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
        plot_multiple_images(generated_images, 8)                     # not shown
        plt.show()                                                    # not shown

In [10]:
generator = keras.Sequential([
  keras.layers.Dense(256 * 12 * 12, activation = "selu", input_shape = [ENCODING_SIZE]),
  keras.layers.Reshape([12, 12, 256]),
  keras.layers.BatchNormalization(),
  keras.layers.Conv2DTranspose(filters = 256, kernel_size = 3, strides = 2,
                               padding = "same", activation = "selu"),
  keras.layers.BatchNormalization(),
  keras.layers.Conv2DTranspose(filters = 128, kernel_size = 3, strides = 2,
                               padding = "same", activation = "selu"),
  keras.layers.BatchNormalization(),
  keras.layers.Conv2DTranspose(filters = 64, kernel_size = 3, strides = 2,
                               padding = "same", activation = "selu"),
  keras.layers.Conv2DTranspose(filters = 3, kernel_size = 3, strides = 1,
                               padding = 'same', activation = 'tanh')
])
generator.summary()   

discriminator = keras.Sequential([
  keras.layers.Conv2D(64, input_shape = [IMG_WIDTH, IMG_HEIGHT, 3], kernel_size = 3, 
                      strides = 1, padding = 'same', activation = 'selu',
                      kernel_initializer='lecun_normal'),
  keras.layers.Conv2D(128, kernel_size = 3, strides = 2,
                      padding = 'same', activation = 'selu',
                      kernel_initializer='lecun_normal'),
  keras.layers.Conv2D(256, kernel_size = 3, strides = 2,
                      padding = 'same', activation = 'selu',
                      kernel_initializer='lecun_normal'),
  keras.layers.Flatten(),
  keras.layers.Dense(1, activation = 'sigmoid')                                  
])
discriminator.summary()

gan = keras.models.Sequential([generator, discriminator])
gan.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 36864)             331776    
                                                                 
 reshape_6 (Reshape)         (None, 12, 12, 256)       0         
                                                                 
 batch_normalization_23 (Bat  (None, 12, 12, 256)      1024      
 chNormalization)                                                
                                                                 
 conv2d_transpose_26 (Conv2D  (None, 24, 24, 256)      590080    
 Transpose)                                                      
                                                                 
 batch_normalization_24 (Bat  (None, 24, 24, 256)      1024      
 chNormalization)                                                
                                                     